---
# Section 1.4: Positive Definite Systems; Cholesky Decomposition
---

After triangular linear systems, the next easiest type of linear system to solve is the **symmetric positive definite** linear system.

A matrix $A \in \mathbb{R}^{n \times n}$ is **symmetric** if $A = A^T$.

For example,

$$
A = 
\begin{bmatrix}
1 & 2 & 3 \\
2 & 4 & 5 \\
3 & 5 & 6
\end{bmatrix}
$$

is a symmetric matrix.

A matrix $A \in \mathbb{R}^{n \times n}$ is called **positive definite** if $A$ is symmetric and 

$$
x^T A x > 0, \qquad \text{for all nonzero $x \in \mathbb{R}^n$.}
$$

---

### Exercise:

Let $A = \begin{bmatrix} 2 & -1 \\ -1 & 2 \end{bmatrix}$ and $x = \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \neq 0$. Prove that $x^T A x > 0$.

---

## Properties of positive definite matrices

1. If $A$ is positive definite, then $A$ is nonsingular.
2. If $A$ is positive definite, then $Ax = b$ has a unique solution.
3. If $A = M^TM$ for some nonsingular $M \in \mathbb{R}^{n \times n}$, then $A$ is positive definite.
4. $A$ is positive definite $\Longleftrightarrow$ all eigenvalues of $A$ are positive.

---
### Exercise:

Prove item 1.

---
### Exercise:

Based on item 3, write a `Julia` function `A = randspd(n)` that randomly generates an $n \times n$ symmetric positive definite matrix $A$. Check that the matrix $A$ is positive definite by computing its eigenvalues.

---

## The Cholesky Decomposition

It turns out that every positive definite matrix $A$ can be written as $A = M^TM$, for some nonsingular matrix $M$.

> ### Cholesky Decomposition Theorem
> Let $A$ be positive definite.
> Then there is a **unique upper-triangular matrix** $R$ with **positive diagonal entries** such that
> $$A = R^T R.$$

We will see a proof of this theorem later.

---

### Exercise:

Computing the Cholesky factor of a random positive definite matrix $A$ using the `Julia` function `cholesky`. Check that the output is correct.

---

### Exercise:

The function `cholesky` will detect if $A$ is not positive definite. In fact, this is the **most efficient** method for testing if a symmetric matrix is positive definite or not.

See what happens when you give `cholesky` a matrix that is symmetric but not positive definite.

---

### Using the Cholesky factor of $A$ to solve $Ax = b$

If we know the **Cholesky factor** $R$ of a positive definite matrix $A$ (that is, $A = R^TR$, where $R$ is upper-triangular with positive diagonal entries), we can solve $Ax = b$ in $O(n^2)$ operations.

First, notice that if $A = R^TR$ and $Ax = b$, then

$$R^T R x = b.$$

Letting $y = Rx$, we have $R^T y = b$. Thus, we have the following algorithm for solving $Ax = b$.

#### Algorithm

1. Solve $R^Ty = b$ for $y$.
2. Solve $Rx = y$ for $x$.

#### Flop count

We can solve the **lower-triangular** system $R^T y = b$ using **forward-substitution** in $n^2$ operations.

We can solve the **upper-triangular** system $R x = y$ using **backward-substitution** in $n^2$ operations.

Thus, we can use the Cholesky factor of $A$ to solve $Ax = b$ in

$$
n^2 + n^2 = 2n^2 = O(n^2)
$$

operations.


---

### Example

In [30]:
n = 5
M = randn(n, n)
A = M'*M
b = randn(n)

5-element Array{Float64,1}:
  0.980697
  1.83942 
  0.965977
 -0.195124
  1.30935 

In [31]:
R = chol(A)

5x5 UpperTriangular{Float64,Array{Float64,2}}:
 2.08315  1.83177  0.820076   0.891675  -0.389599
 0.0      1.55435  0.478399  -0.852767   0.701642
 0.0      0.0      2.49134   -0.992042   0.813113
 0.0      0.0      0.0        1.57776    0.816253
 0.0      0.0      0.0        0.0        1.50113 

In [32]:
# Solve R'*y = b
y = R'\b

5-element Array{Float64,1}:
 0.470775 
 0.628604 
 0.112061 
 0.0204849
 0.628771 

In [33]:
# Solve R*x = y
x = R\y

5-element Array{Float64,1}:
  0.321719
  0.156771
 -0.172846
 -0.203716
  0.418865

In [34]:
# In one line
x = R\(R'\b)

5-element Array{Float64,1}:
  0.321719
  0.156771
 -0.172846
 -0.203716
  0.418865

In [35]:
A*x - b

5-element Array{Float64,1}:
  1.11022e-16
 -2.22045e-16
 -1.11022e-16
 -1.38778e-16
 -4.44089e-16

---

## How to calculate the Cholesky factor

### Example

We begin by writing $A = R^T R$, where $R$ is **upper-triangular**:

$$ 
\begin{bmatrix}
4 & -2 & 4 & 2 \\
-2 & 10 & -2 & -7 \\
4 & -2 & 8 & 4 \\
2 & -7 & 4 & 7
\end{bmatrix} =
\begin{bmatrix}
r_{11} \\
r_{12} & r_{22} \\
r_{13} & r_{23} & r_{33} \\
r_{14} & r_{24} & r_{34} & r_{44} \\
\end{bmatrix}
\begin{bmatrix}
r_{11} & r_{12} & r_{13} & r_{14} \\
       & r_{22} & r_{23} & r_{24} \\
       &        & r_{33} & r_{34} \\
       &        &        & r_{44} \\
\end{bmatrix}
$$

---

### General Cholesky Formula

Based on the above example, we can obtain the general formulas for the entries $r_{ij}$ of the Cholesky factor $R$ of a positive definite matrix $A$.

$$
\begin{align}
r_{ii} &= \sqrt{a_{ii} - \sum_{k=1}^{i-1} r_{ki}^2}, \\
r_{ij} &= \left(a_{ij} - \sum_{k=1}^{i-1} r_{ki} r_{kj}\right)\bigg/r_{ii}, \qquad j = i+1, \ldots, n. \\
\end{align}
$$

From these formulas, we can see that if $A = R^TR$, where $R$ is an upper-triangular matrix $R$ with positive diagonals, then $R$ is **uniquely determined**.

---

### Cholesky's Algorithm (inner-product form)

In [1]:
"""
 CHOL_IP   Inner-product form of Cholesky's algorithm
    R = CHOL_IP(A) computes the Cholesky factor of A using the
    inner-product version of Cholesky's algorithm.
"""
function chol_ip(A::Array{Float64,2})

    # Check that A is a square and symmetric matrix
    issym(A) || error("Matrix must be symmetric.")
        
    # Initialize R to be the upper-triangular part of A
    R = triu(A)

    n = size(A, 1)

    for i = 1:n

        # R[i,i] = sqrt(A[i,i] - sum_{k=1}^{i-1} R[k,i]^2)
        for k = 1:i-1
            R[i,i] -= R[k,i]*R[k,i]
        end

        # Check that R[i,i] is positive before taking square root
        if R[i,i] > 0
            R[i,i] = sqrt(R[i,i])
        else
            # If R[i,i] <= 0, then A is not positive definite
            warn("Matrix is not positive definite.")
            return R
        end

        # R[i,j] = (A[i,j] - sum_{k=1}^{i-1} R[k,i]*R[k,j]) / R[i,i]
        for j = i+1:n
            for k = 1:i-1
                R[i,j] -= R[k,i]*R[k,j]
            end
            R[i,j] /= R[i,i]
        end

    end
    
    return R
end


chol_ip (generic function with 1 method)

In [41]:
A = [4 -2 4 2; -2 10 -2 -7; 4 -2 8 4; 2 -7 4 7]
R = chol_ip(A)

LoadError: LoadError: MethodError: `chol_ip` has no method matching chol_ip(::Array{Int64,2})
while loading In[41], in expression starting on line 2

In [42]:
A = Float64[4 -2 4 2; -2 10 -2 -7; 4 -2 8 4; 2 -7 4 7]
R = chol_ip(A)

4x4 Array{Float64,2}:
 2.0  -1.0  2.0   1.0
 0.0   3.0  0.0  -2.0
 0.0   0.0  2.0   1.0
 0.0   0.0  0.0   1.0

In [2]:
A = rand(3, 3)
R = chol_ip(A)

LoadError: LoadError: Matrix must be symmetric.
while loading In[2], in expression starting on line 2

In [3]:
A = rand(3, 3)
R = chol_ip(A + A')

3x3 Array{Float64,2}:
 0.532919    3.35186  1.81124
 0.0       -10.2648   1.01583
 0.0         0.0      1.55676

---

### Flop count for Cholesky's Algorithm

```julia
for i = 1:n
    for k = 1:i-1
        R[i,i] -= R[k,i]*R[k,i]
    end

    R[i,i] = sqrt(R[i,i])

    for j = i+1:n
        for k = 1:i-1
            R[i,j] -= R[k,i]*R[k,j]
        end
        R[i,j] /= R[i,i]
    end
end
```

Therefore, we can calculate the total number of flops, $T$, as follows:

$$
T = \sum_{i=1}^n \left( \sum_{k=1}^{i-1} 2 + 1 + \sum_{j=i+1}^n \left(\sum_{k=1}^{i-1} 2 + 1 \right) \right) 
= -2 \sum_{i=1}^n i^2 + (2n + 3) \sum_{i=1}^n i - n(n+1).
$$

Now we use the formulas:

$$
\sum_{i=1}^n i^2 = \frac{n(n+1)(2n+1)}{6} = \frac{n^3}{3} + O(n^2), \qquad \sum_{i=1}^n i = \frac{n(n+1)}{2} = \frac{n^2}{2} + O(n).
$$

An easy way to get these estimates is by using integration. For example,

$$
\int_0^n x^2 dx \leq \sum_{i=0}^n i^2 \leq \int_0^{n+1} x^2 dx,
$$

which implies that 

$$
\frac{n^3}{3} \leq \sum_{i=0}^n i^2 \leq \frac{(n+1)^3}{3}
$$

and so

$$
\fbox{$\sum_{i=0}^n i^2 = \frac{n^3}{3} + O(n^2)$.}
$$

Therefore,

$$
T = \frac{n^3}{3} + O(n^2).
$$

Therefore, computing the Cholesky factor $R$ of a positive definite matrix $A$ takes approximately $\frac{n^3}{3}$ flops.

---

Thus, to solve $Ax=b$ for a symmetric positive definite matrix $A$, we do this:

1. `R = chol(A)`
2. Solve the upper-triangular system $R^Ty = b$ for $y$ using backward-substitution
3. Solve the lower-triangular system $Rx = y$ for $x$ using forward-substitution

which requires

$$
\frac{n^3}{3} + O(n^2) + n^2 + n^2 = \fbox{$\frac{n^3}{3} + O(n^2)$}
$$

flops in total.

---

## Outer-Product Form of Cholesky's Algorithm

Partition $A = R^TR$ as:

$$
\begin{bmatrix}
a_{11} & b^T \\
b & \hat{A} \\
\end{bmatrix} 
= 
\begin{bmatrix}
r_{11} & 0 \\
s & \hat{R}^T \\
\end{bmatrix}
\begin{bmatrix}
r_{11} & s^T \\
0 & \hat{R} \\
\end{bmatrix}
$$

Then:

$$
\begin{bmatrix}
a_{11} & b^T \\
b & \hat{A} \\
\end{bmatrix} 
= 
\begin{bmatrix}
r_{11}^2 & r_{11}s^T \\
r_{11}s & ss^T + \hat{R}^T\hat{R} \\
\end{bmatrix}
$$

Therefore,

$$
a_{11} = r_{11}^2, \qquad b^T = r_{11}s^T, \qquad \hat{A} = ss^T + \hat{R}^T\hat{R}.
$$

This gives us the following **recursive algorithm**:

> To compute the Cholesky factor $R$ of $A$:
1. Let $r_{11} = +\sqrt{a_{11}}$.
2. If $n > 1$:
    - Let $s^T = b^T/r_{11}$.
    - Compute the Cholesky factor $\hat{R}$ of $\hat{A} - ss^T$.
    
It is not too hard to develop a nonrecursive version of this algorithm.

In [1]:
M = triu(round(4*rand(3,3)))
A = M'*M

3x3 Array{Float64,2}:
 16.0  12.0   8.0
 12.0  18.0  15.0
  8.0  15.0  17.0

In [2]:
chol(A)

3x3 UpperTriangular{Float64,Array{Float64,2}}:
 4.0  3.0  2.0
 0.0  3.0  3.0
 0.0  0.0  2.0

In [5]:
A

3x3 Array{Float64,2}:
  4.0   3.0  2.0
 12.0   3.0  3.0
  8.0  15.0  2.0

In [4]:
cholfact!(A)

Base.LinAlg.Cholesky{Float64,Array{Float64,2}} with factor:
3x3 UpperTriangular{Float64,Array{Float64,2}}:
 4.0  3.0  2.0
 0.0  3.0  3.0
 0.0  0.0  2.0

---

## A recursive Cholesky Algorithm

In [17]:
"""
 CHOL_ROP   Recursive outer-product form of Cholesky's algorithm
    R = CHOL_ROP(A) computes the Cholesky factor of A using the
    recursive outer-product version of Cholesky's algorithm.
"""
function chol_rop(A::Array{Float64,2})

    # Check that A is a square and symmetric matrix
    issym(A) || error("Matrix must be symmetric.")
        
    # Initialize R = 0 and the same size and datatype as A
    R = zeros(A)
    
    n = size(A, 1)

    # Make sure A[1,1] is positive before taking the square root
    if A[1,1] <= 0.0
        # If A[1,1] <= 0.0, then A is not positive definite
        error("Matrix is not positive definite.")
    else
        R[1,1] = sqrt(A[1,1])
    end

    if n > 1
        # Extract b and Ahat from A
        b = A[2:n,1]
        Ahat = A[2:n,2:n]

        # Compute s and Rhat
        s = b/R[1,1]
        Rhat = chol_rop(Ahat - s*s')

        # Form R
        R[1,2:n] = s'
        R[2:n,2:n] = Rhat
    end
    
    return R
    
end

chol_rop (generic function with 1 method)

In [18]:
A = Float64[4 -2 4 2; -2 10 -2 -7; 4 -2 8 4; 2 -7 4 7]
R = chol_rop(A)

4x4 Array{Float64,2}:
 2.0  -1.0  2.0   1.0
 0.0   3.0  0.0  -2.0
 0.0   0.0  2.0   1.0
 0.0   0.0  0.0   1.0

---

## Bordered Form of Cholesky's Algorithm

Partition $A = R^TR$ as:

$$
\begin{bmatrix}
\hat{A} & c \\
c^T & a_{nn} \\
\end{bmatrix} 
= 
\begin{bmatrix}
\hat{R}^T & 0 \\
h^T & r_{nn} \\
\end{bmatrix}
\begin{bmatrix}
\hat{R} & h \\
0 & r_{nn} \\
\end{bmatrix}
$$

Then:

$$
\begin{bmatrix}
\hat{A} & c \\
c^T & a_{nn} \\
\end{bmatrix} 
= 
\begin{bmatrix}
\hat{R}^T\hat{R} & \hat{R}^T h \\
h^T\hat{R} & h^T h + r_{nn}^2 \\
\end{bmatrix}
$$

Therefore,

$$
\hat{R}^T\hat{R} = \hat{A}, \qquad \hat{R}^T h = c, \qquad h^T h + r_{nn}^2 = a_{nn}
$$

This gives us the following **recursive algorithm**:

> To compute the Cholesky factor $R$ of $A$:
>
- If $n = 1$, let $R = \sqrt{A}$; otherwise:
    1. Compute the Cholesky factor $\hat{R}$ of $\hat{A}$.
    2. Solve the lower-triangular system $\hat{R}^T h = c$ for $h$.
    3. Let $r_{nn} = +\sqrt{a_{nn} - h^T h}$.

---

## Proof of the Cholesky Decomposition Theorem

Recall:

> ### Cholesky Decomposition Theorem
> Let $A$ be positive definite.
> Then there is a **unique upper-triangular matrix** $R$ with **positive diagonal entries** such that
> $$A = R^T R.$$

We will prove this theorem using mathematical induction. First we need to prove a couple of lemmas.

> ### Lemma 1
> Let $A$ be positive definite and partitioned as
$$
A = 
\begin{bmatrix}
A_{11} & A_{12} \\
A_{21} & A_{22}
\end{bmatrix}
$$
where $A_{11} \in \mathbb{R}^{n_1 \times n_1}$ and $A_{22} \in \mathbb{R}^{n_2 \times n_2}$. Then $A_{11}$ and $A_{22}$ are positive definite.

> ### Lemma 2
> Let $A, X \in \mathbb{R}^{n \times n}$, with $A$ positive definite and $X$ nonsingular.
Then $B = X^TAX$ is positive definite.

---